In [6]:
import psycopg2
import pandas as pd
import subprocess
import xarray as xr

from getpass import getpass
from subprocess import call

/Users/sato/dev/cc/sandbox-fabio/python-notebooks/env/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Database Connection

Type database password

In [9]:
password = getpass()

········


In [10]:
conn = psycopg2.connect("dbname='ccgeodb' user='linda_ml' host='10.0.18.10' password='%s'" % (password))

# Load CDO Temperature and Precipitation indexes

Open dataset from single netcdf files

In [16]:
files = [
    "../era5/era5-nordics-ecacfd.nc",
    "../era5/era5-nordics-ecafd.nc",
    "../era5/era5-nordics-ecacsu.nc",
    "../era5/era5-nordics-ecaid.nc",
    "../era5/era5-nordics-ecasu.nc",
    "../era5/era5-nordics-ecacdd.nc",
    "../era5/era5-nordics-ecacwd.nc",
    "../era5/era5-nordics-ecapd.nc",
    "../era5/era5-nordics-ecarr1.nc",
    "../era5/era5-nordics-ecarx1day.nc",
    "../era5/era5-nordics-ecarx5day.nc",
    "../era5/era5-nordics-ecasdii.nc"
]

# Create list of variables per file

Some files have more than one climate index variable

In [30]:
variables = {}
for f in files:
    ds = xr.open_dataset(f)
    for v in ds.variables:
        if f not in variables:
            variables[f] = []
        if v not in ["time", "longitude", "latitude", "inate.", "oordinate."]:
            variables[f].append(v)

print(variables)

{'../era5/era5-nordics-ecacfd.nc': ['number_of_cfd_periods_with_more_than_5days_per_time_period'], '../era5/era5-nordics-ecafd.nc': ['frost_days_index_per_time_period'], '../era5/era5-nordics-ecacsu.nc': ['number_of_csu_periods_with_more_than_5days_per_time_period'], '../era5/era5-nordics-ecaid.nc': ['ice_days_index_per_time_period'], '../era5/era5-nordics-ecasu.nc': ['summer_days_index_per_time_period'], '../era5/era5-nordics-ecacdd.nc': ['consecutive_dry_days_index_per_time_period', 'number_of_cdd_periods_with_more_than_5days_per_time_period'], '../era5/era5-nordics-ecacwd.nc': ['consecutive_wet_days_index_per_time_period', 'number_of_cwd_periods_with_more_than_5days_per_time_period'], '../era5/era5-nordics-ecapd.nc': ['precipitation_days_index_per_time_period'], '../era5/era5-nordics-ecarr1.nc': ['wet_days_index_per_time_period'], '../era5/era5-nordics-ecarx1day.nc': ['highest_one_day_precipitation_amount_per_time_period'], '../era5/era5-nordics-ecarx5day.nc': ['highest_five_day_pre

Convert each one of the variables into single GeoTiff files

In [36]:
for f in files:
    print(f)
    for v in variables[f]:
        g = "%s.tiff" % v
        print(g)
        call(["gdal_translate", "-a_srs", "EPSG:4258", "NETCDF:%s:%s" % (f, v), 
              "-of", "GTiff", "%s" % g])

../era5/era5-nordics-ecacfd.nc
number_of_cfd_periods_with_more_than_5days_per_time_period.tiff
../era5/era5-nordics-ecafd.nc
frost_days_index_per_time_period.tiff
../era5/era5-nordics-ecacsu.nc
number_of_csu_periods_with_more_than_5days_per_time_period.tiff
../era5/era5-nordics-ecaid.nc
ice_days_index_per_time_period.tiff
../era5/era5-nordics-ecasu.nc
summer_days_index_per_time_period.tiff
../era5/era5-nordics-ecacdd.nc
consecutive_dry_days_index_per_time_period.tiff
number_of_cdd_periods_with_more_than_5days_per_time_period.tiff
../era5/era5-nordics-ecacwd.nc
consecutive_wet_days_index_per_time_period.tiff
number_of_cwd_periods_with_more_than_5days_per_time_period.tiff
../era5/era5-nordics-ecapd.nc
precipitation_days_index_per_time_period.tiff
../era5/era5-nordics-ecarr1.nc
wet_days_index_per_time_period.tiff
../era5/era5-nordics-ecarx1day.nc
highest_one_day_precipitation_amount_per_time_period.tiff
../era5/era5-nordics-ecarx5day.nc
highest_five_day_precipitation_amount_per_time_perio

Generate SQL script to load each raster

In [41]:
cursor = conn.cursor()
for f in files:
    for v in variables[f]:
        g = "%s.tiff" % v
        table = "era5_nordics_%s" % g[0:-21]
        print(g)
        print(table)
        output = subprocess.check_output(["raster2pgsql", "-I", "%s" % g, 
                                          "climate.%s" % table])

        output = output.decode('utf-8')
        statements = output.split('\n')

        # skip BEGIN and COMMIT statements
        for i in range(1, 4):
            if i == 1:
                print(statements[i])
            cursor.execute(statements[i])

        conn.commit()

number_of_cfd_periods_with_more_than_5days_per_time_period.tiff
number_of_cfd_periods_with_more_than_5days
CREATE TABLE "climate"."number_of_cfd_periods_with_more_than_5days" ("rid" serial PRIMARY KEY,"rast" raster);


KeyboardInterrupt: 